<a href="https://colab.research.google.com/github/kachytronico/colab-PIA/blob/main/2025ORD1_Ruben.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importación de datos

In [ ]:
!git clone https://github.com/ruescog/datasets

fatal: destination path 'datasets' already exists and is not an empty directory.


In [ ]:
!unzip datasets/PIA/2025/ORD1/P1.zip

Archive:  datasets/PIA/2025/ORD1/P1.zip
  inflating: test.csv                
  inflating: train.csv               


#Librerias

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

random_seed = 33

# AED

In [ ]:
train = pd.read_csv("train.csv")

In [ ]:
train

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


In [ ]:
train.describe()

In [ ]:
train.corr()

# Prepro

# Hito 1

In [ ]:
train, val = train_test_split(train, test_size=0.15)

# Hito 2

In [ ]:
train.columns

In [ ]:
train["wifi"] = train["wifi"].apply(lambda e: e if random.random() < 0.9 else None)

In [ ]:
train.info()

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="most_frequent")
imputer.fit(train[["wifi"]])
imputer.transform(train[["wifi"]])

El conjunto de datos no tiene valores raros ni nulos.

# Hito 3

El conjunto de datos no tiene variables categóricas.

# Hito 4

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

Usaré esto en mi función de limpieza.

## Función de limpieza

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Normalizer

In [ ]:
train = pd.read_csv("train.csv")
train, val = train_test_split(train, test_size=0.15)

In [ ]:
def clean(df, test = None):
  # columnas
  if not test.empty and "id" in test.columns:
    test = test.drop(columns=["id"])

  # nulos
  imputer = SimpleImputer(strategy="most_frequent")
  imputer = imputer.fit(df[["wifi"]])
  df["wifi"] = imputer.transform(df[["wifi"]])

  if not test.empty:
    test["wifi"] = imputer.transform(test[["wifi"]])

  # variables categóricas
  # no hay

  return df, test if not test.empty else None

def train_test(model, X_train, y, X_test = None):
  # normalización
  normalizer = Normalizer()
  normalizer = normalizer.fit(X_train)
  X_train = normalizer.transform(X_train)

  if not X_test.empty:
    X_test = normalizer.transform(X_test)

  # dimensionalidad
  sfs = SequentialFeatureSelector(model)
  sfs = sfs.fit(X_train, y)
  X_train = sfs.transform(X_train)

  if X_test.any():
    X_test = sfs.transform(X_test)

  # entrenamiento
  model.fit(X_train, y)
  train_preds = model.predict(X_train)

  if X_test.any():
    preds = model.predict(X_test)

  return model, train_preds, preds if X_test.any() else None

def evaluate(real_y, preds):
  return accuracy_score(real_y, preds)

In [ ]:
train, val = clean(train, val)

In [ ]:
X_train, y_train = train.drop(columns = "price_range"),  train["price_range"]
X_val, y_val = val.drop(columns = "price_range"), val["price_range"]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model, train_preds, val_preds = train_test(DecisionTreeClassifier(), X_train, y_train, X_val)

In [ ]:
evaluate(y_train, train_preds)

In [ ]:
evaluate(y_val, val_preds)

## Optimización

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
model = DecisionTreeClassifier(random_state=random_seed)
param_grid = {
    "criterion": ["gini", "entropy", "log_loss"],
    "max_depth": range(2, 7),
    "min_samples_split": range(2, 10)
}

In [ ]:
gs = GridSearchCV(model, param_grid=param_grid, cv=5) # cv son los folds
gs.fit(X_train, y_train);

In [ ]:
gs.best_estimator_

In [ ]:
model, train_preds, val_preds = train_test(gs.best_estimator_, X_train, y_train, X_val)

In [ ]:
evaluate(y_train, train_preds)

In [ ]:
evaluate(y_val, val_preds)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
model, train_preds, val_preds = train_test(KNeighborsClassifier(), X_train, y_train, X_val)

In [ ]:
evaluate(y_train, train_preds)

In [ ]:
evaluate(y_val, val_preds)

In [ ]:
# cogería dos modelos de la librería TAL y los optimizaría usando una estrategia de GridSearch...